# Assignment 3

Imports

In [ ]:
import numpy as np
import scipy as sp
from scipy import stats as st
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pandas as pd
from tqdm import tqdm
from numba import jit, njit, prange
import math
import os
import sys

## General stuff about Lotka-Volterra equations

any visualizations we need

## Objective functions